In [1]:
import random
from math import sqrt, gcd
import numpy as np

# 81 - Path Sum: Two Ways

In [2]:
%%time
def pe_81():
    with open("./inputs/0081_matrix.txt", 'r') as file:
        matrix = [[int(x) for x in line.strip().split(',')] for line in file]
    m_len = len(matrix)
    triangle = [[] for _ in range(2*m_len-1)]
    for i in range(m_len):
        for j in range(m_len):
            triangle[i+j].append(matrix[i][j])
    half = len(triangle)//2 +1
    for i in range(1, half):
        r_len = len(triangle[i])
        triangle[i][0] += triangle[i-1][0]
        triangle[i][r_len-1] += triangle[i-1][r_len-2]
        for j in range(1, r_len-1):
            a = triangle[i-1][j]
            b = triangle[i-1][j-1]
            triangle[i][j] = triangle[i][j] + a if a < b else triangle[i][j] + b
    for i in range(half,len(triangle)):
        for j in range(0, len(triangle[i])):
            a = triangle[i-1][j]
            b = triangle[i-1][j+1]
            triangle[i][j] = triangle[i][j] + a if a < b else triangle[i][j] + b
    print(triangle[-1][0])     
pe_81()

427337
CPU times: user 2.5 ms, sys: 285 μs, total: 2.78 ms
Wall time: 2.37 ms


# 82 - Path Sum: Three Ways

In [3]:
%%time
def pe_82():
    with open("./inputs/0082_matrix.txt", 'r') as file:
        matrix = [[int(x) for x in line.strip().split(',')] for line in file]
    m_len = len(matrix)
    dp = [row[:] for row in matrix]
    for c in range(1, m_len): 
        for r in range(m_len):
            min_sum = matrix[r][c] + dp[r][c-1]
            for i in range(m_len):
                c_sum = dp[i][c-1] + matrix[i][c]
                j=i
                while j != r:
                    j = j-1 if j>r else j+1
                    c_sum += matrix[j][c]
                if c_sum < min_sum: min_sum = c_sum
            dp[r][c] = min_sum
    print(min([row[-1] for row in dp]))
pe_82()

260324
CPU times: user 627 ms, sys: 7.46 ms, total: 635 ms
Wall time: 633 ms


# 83 - Path Sum: Four Ways

In [4]:
class Node:
    def __init__(self, x, y, weight):
        self.pos = (x , y)
        self.weight = weight
    def __repr__(self):
        return f"{self.weight}"

In [5]:
%%time
def pe_83():
    with open("./inputs/0083_matrix.txt", 'r') as file:
        matrix = [[int(x) for x in line.strip().split(',')] for line in file]
    m_len = len(matrix)
    dp = [[0 for _ in range(m_len)] for _ in range(m_len)]
    unvisited = []
    for i in range(m_len):
        for j in range(m_len):
            temp_node = Node(i,j,float('inf'))
            dp[i][j] = temp_node
            unvisited.append(temp_node)        
    curr_node = unvisited[0]
    curr_node.weight = matrix[0][0]
    while unvisited:
        for step in [(0,1) , (0,-1) , (-1,0) , (1,0)]:
            x_1 , y_1 = tuple(a + b for a, b in zip(curr_node.pos, step))
            if x_1 < 0 or y_1<0 or x_1 > m_len-1 or y_1 > m_len-1: continue
            temp_node = dp[x_1][y_1]
            new_weight = curr_node.weight + matrix[x_1][y_1]
            if new_weight<temp_node.weight:
                temp_node.weight = new_weight    
        unvisited = unvisited[1:]
        unvisited = sorted(unvisited, key = lambda n: n.weight)
        if unvisited: curr_node = unvisited[0]
    print(dp[m_len-1][m_len-1])
pe_83()

425185
CPU times: user 740 ms, sys: 4.15 ms, total: 744 ms
Wall time: 743 ms


# 84 - Monopoly Odds

In [6]:
%%time
def pe_84():
    landed_dict = {i:0 for i in range(40)}
    cc_dict = { 0:0 , 1:10 }
    ch_dict = { 0:0 , 1:10 , 2:11 , 3:24 , 4:39 , 5:5 }
    ch_r_dict = {7:15 , 22:25 , 36:5 }
    ch_u_dict = {7:12 , 22:28 , 36:12}
    player_pos=0
    doubles=0
    cc=0
    ch=0
    for _ in range(10**6):
        die_roll_1 = random.randint(1, 4)
        die_roll_2 = random.randint(1, 4)
        if die_roll_1 == die_roll_2:
            doubles = (doubles + 1) % 3
            if not doubles:
                player_pos = 10
                landed_dict[player_pos] += 1
                continue
        else:
            doubles = 0
        player_pos = (player_pos + die_roll_1 + die_roll_2)%40
        if player_pos == 30:
            player_pos = 10
        if player_pos in [7, 22, 36]:
            if ch in ch_dict:
                player_pos = ch_dict[ch]
            elif ch in [6,7]:
                player_pos = ch_r_dict[player_pos]
            elif ch == 8:
                player_pos = ch_u_dict[player_pos]
            elif ch == 9:
                player_pos = player_pos - 3
            ch = (ch+1)%16
        if player_pos in [2, 17, 33]:
            if cc in cc_dict: player_pos = cc_dict[cc]
            cc = (cc + 1)%16
        landed_dict[player_pos] += 1
    modal_string = ""
    top_squares = [(k,v) for k, v in sorted(landed_dict.items(), key=lambda item: item[1], reverse=True)[:3]]
    for val in top_squares:
        #print(f"{val[0]}: {val[1]}")
        modal_string += str(val[0])
    print(modal_string)
pe_84()

101524
CPU times: user 624 ms, sys: 73 μs, total: 624 ms
Wall time: 624 ms


# 85 - Counting Rectangles

In [7]:
def count_rectangles(n, m):
    total = 0
    for i in range(1,n+1):
        for j in range(1,m+1):
            total += (n-i+1)*(m-j+1)
    return total

In [8]:
%%time
def pe_85():
    min_dist = (0, float('inf'))
    for n in range(100):
        for m in range(100):
            count = count_rectangles(n,m)
            dist = abs((2*(10**6)) - count)
            if dist < min_dist[1]: min_dist = (n*m, dist)
    print(min_dist[0])
pe_85()

2772
CPU times: user 960 ms, sys: 496 μs, total: 961 ms
Wall time: 959 ms


# 86 - Cuboid Route

In [9]:
def combinations(a, b_c):
    if 2*a < b_c: return 0
    if a >= b_c: return b_c // 2
    return a - (b_c - 1)//2

In [10]:
%%time
def pe_86():
    limit = 10**6
    m_limit = int(sqrt(2*limit))
    combinations_dict = {i:0 for i in range(2*limit)}
    for i in range(1,m_limit):
        for j in range(i+1,m_limit):
            if not (i%2 ^ j%2) or gcd(i,j)!=1: continue
            x = int(j**2 - i**2)
            y = int(2*i*j)
            for k in range(1, limit):
                if k*x > limit: break
                combinations_dict[k*x] += combinations(k*x, k*y)
            for k in range(1, limit):
                if k*y > limit: break
                combinations_dict[k*y] += combinations(k*y, k*x)
    total = 0
    for k, v in combinations_dict.items():
        total += v
        if total > limit:
            print(k)
            return
pe_86()

1818
CPU times: user 4.03 s, sys: 56 ms, total: 4.08 s
Wall time: 4.08 s


# 87 - Prime Power Triples

In [11]:
def prime_array(size):
    prime_array = np.ones(size, np.bool_)
    prime_array[0] = False
    prime_array[1] = False
    sqrt_size = int(sqrt(size)) + 1
    for i in range(2,sqrt_size):
        if prime_array[i]:
            prime_array[i*i:size:i] = False
    return prime_array

In [12]:
%%time
def pe_87():
    limit = 50 * 10**6
    primes = prime_array(limit)
    fourths = [i**4 for i in range(2, int(sqrt(sqrt(limit))) + 1) if primes[i]]
    cubes = [i**3 for i in range(2, int(sqrt(limit)) + 1) if i**3 < limit and primes[i]]
    squares = [i**2 for i in range(2, int(sqrt(limit)) + 1) if primes[i]]
    nums = set()
    for f in fourths:
        for c in cubes:
            if f+c > limit: break
            for s in squares:
                sum_n = f + c + s
                if sum_n > limit: break
                nums.add(sum_n)
    print(len(nums))
pe_87()

1097343
CPU times: user 355 ms, sys: 24 ms, total: 379 ms
Wall time: 377 ms


# 88 - Product-sum Numbers

In [13]:
def factors(l_f, p_d, s_m, l_n, limit, data):
    if p_d > 2*limit: return
    if l_n != 1:
        k = p_d - s_m + l_n
        if k > limit: return
        if p_d < data[k]: data[k] = p_d
    for q in range(l_f, (2*limit)//p_d +1): factors(q, p_d*q, s_m+q, l_n+1, limit, data)

In [14]:
%%time
def pe_88():
    limit = 12000
    data = {i:float('inf') for i in range(2, limit+1)}
    for i in range(2, limit+1): factors(i, i, i, 1, limit, data)
    print(sum(set(list(data.values()))))
pe_88()

7587457
CPU times: user 95 ms, sys: 3.97 ms, total: 99 ms
Wall time: 97.1 ms


# 89 - Roman Numerals

In [15]:
%%time
def pe_89():
    num_to_int = {
        'I': 1,
        'V': 5,
        'X': 10,
        'L': 50,
        'C': 100,
        'D': 500,
        'M': 1000
    }
    int_to_num = {v:k for k,v in num_to_int.items()}
    with open("./inputs/0089_roman.txt", 'r') as f:
        numbers = [[c for c in line.strip()] for line in f]
    chars_saved = 0
    for num in numbers:
        start_len = len(num)
        total = 0
        while num:
            c = num.pop()
            prev = num[-1] if num else None
            if (prev == 'I' and c in ['V', 'X']) or (prev == 'X' and c in ['L', 'C']) or (prev == 'C' and c in ['D', 'M']):
                total += num_to_int[c] - num_to_int[prev]
                num.pop()
            else:
                total += num_to_int[c]
        min_str = ""
        thousands = total // 1000 
        total = total % 1000
        for _ in range(thousands): min_str += 'M'
        for place in [100, 10, 1]:
            place_value = total // place
            total = total % place
            if 0 <= place_value <= 3:
                for _ in range(place_value): min_str += int_to_num[place]
            elif place_value == 4:
                min_str += int_to_num[place]
                min_str += int_to_num[place*5]
            elif 5 <= place_value <= 8:
                min_str += int_to_num[place*5]
                for _ in range(place_value - 5): min_str += int_to_num[place]
            else:
                min_str += int_to_num[place]
                min_str += int_to_num[place*10]
        chars_saved += start_len - len(min_str)
    print(chars_saved)
pe_89()

743
CPU times: user 2.24 ms, sys: 0 ns, total: 2.24 ms
Wall time: 2.27 ms


# 90 - Cube Digit Pairs

In [16]:
def cube(digits, digits_list):
    if len(digits) == 6:
        digits_list.append(digits)
        return
    for q in range(digits[-1]+1, 10):cube(digits + [q], digits_list)

In [17]:
def valid_pair(digits_1, digits_2):
    squares = [[0,1], [0,4], [0,6], [1,6], [2,5], [3,6], [4,6], [6,4], [8,1]]
    for square in squares:
        num_1 = square[0]
        num_2 = square[1]
        if num_1 != 6 and num_2 != 6:
            if not ((num_1 in digits_1 and num_2 in digits_2) or (num_1 in digits_2 and num_2 in digits_1)):
                return False
        elif num_1 == 6:
            if not (((6 in digits_1 or 9 in digits_1) and num_2 in digits_2) or ((6 in digits_2 or 9 in digits_2) and num_2 in digits_1)):
                return False
        elif num_2 == 6:
            if not ((num_1 in digits_1 and (6 in digits_2 or 9 in digits_2)) or (num_1 in digits_2 and (6 in digits_1 or 9 in digits_1))):
                return False
    return True

In [18]:
%%time
def pe_90():
    digits_list = []
    total = 0
    for i in range(10):cube([i], digits_list)
    for i,digits_1 in enumerate(digits_list):
        for digits_2 in digits_list[i:]:
            if valid_pair(digits_1, digits_2):
                total += 1
    print(total)
pe_90()

1217
CPU times: user 35.8 ms, sys: 4.11 ms, total: 39.9 ms
Wall time: 38.7 ms
